In [1]:
! pip install -q pyspark==3.2.0 spark-nlp

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType, DataType,ArrayType
from pyspark.sql.functions import udf, struct
from IPython.core.display import display, HTML
import re

/var/folders/pm/yd1j81s52gj2mndwp9tf84n80000gn/T/ipykernel_36462/661913275.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
from pyspark.ml import PipelineModel
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp

spark = sparknlp.start(spark32 = True)

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

spark

22/03/16 01:59:33 WARN Utils: Your hostname, MacBook-Air-Muhammad.local resolves to a loopback address: 127.0.0.1; using 192.168.43.150 instead (on interface en0)
22/03/16 01:59:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/mhihsan/miniconda3/envs/kuliah/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mhihsan/.ivy2/cache
The jars for the packages stored in: /Users/mhihsan/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-spark32_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-481fae93-132b-4a98-9bd7-3eb91bce3e76;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-spark32_2.12;3.4.1 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found or

Spark NLP version 3.4.1
Apache Spark version: 3.2.0


In [4]:
stopwords = StopWordsCleaner().getStopWords()

In [5]:
stopwords[:5]

['i', 'me', 'my', 'myself', 'we']

## YAKE Keyword Extractor

In [8]:
document = DocumentAssembler() \
    .setInputCol("Transcript") \
    .setOutputCol("document")

sentenceDetector = SentenceDetector() \
    .setInputCols("document") \
    .setOutputCol("sentence")

token = Tokenizer() \
    .setInputCols("sentence") \
    .setOutputCol("token") \
    .setContextChars(["(", ")", "?", "!", ".", ","])

keywords = YakeKeywordExtraction() \
    .setInputCols("token") \
    .setOutputCol("keywords") \
    .setMinNGrams(1) \
    .setMaxNGrams(3)\
    .setNKeywords(20)\
    .setStopWords(stopwords)

yake_pipeline = Pipeline(stages=[document, sentenceDetector, token, keywords])

empty_df = spark.createDataFrame([['']]).toDF("Transcript")

yake_Model = yake_pipeline.fit(empty_df)

In [9]:
# LightPipeline

light_model = LightPipeline(yake_Model)

text = """SUBJECTIVE: This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up. MEDICATIONS: Her only medication currently is Ortho Tri-Cyclen and the Allegra. ALLERGIES: She has no known medicine allergies. OBJECTIVE: Vitals: Weight was 130 pounds and blood pressure 124/78. HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear. Neck: Supple without adenopathy. Lungs: Clear. ASSESSMENT: Allergic rhinitis. PLAN: 1. She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper. 2. Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well."""

light_result = light_model.fullAnnotate(text)[0]

[(s.metadata['sentence'], s.result) for s in light_result['sentence']]

[('0',
  'SUBJECTIVE: This 23-year-old white female presents with complaint of allergies.'),
 ('1',
  'She used to have allergies when she lived in Seattle but she thinks they are worse here.'),
 ('2', 'In the past, she has tried Claritin, and Zyrtec.'),
 ('3', 'Both worked for short time but then seemed to lose effectiveness.'),
 ('4', 'She has used Allegra also.'),
 ('5',
  'She used that last summer and she began using it again two weeks ago.'),
 ('6', 'It does not appear to be working very well.'),
 ('7',
  'She has used over-the-counter sprays but no prescription nasal sprays.'),
 ('8',
  'She does have asthma but doest not require daily medication for this and does not think it is flaring up.'),
 ('9',
  'MEDICATIONS: Her only medication currently is Ortho Tri-Cyclen and the Allegra.'),
 ('10', 'ALLERGIES: She has no known medicine allergies.'),
 ('11', 'OBJECTIVE: Vitals: Weight was 130 pounds and blood pressure 124/78.'),
 ('12', 'HEENT: Her throat was mildly erythematous witho

In [10]:
import pandas as pd

keys_df = pd.DataFrame([(k.result, k.begin, k.end, k.metadata['score'],  k.metadata['sentence']) for k in light_result['keywords']],
                       columns = ['keywords','begin','end','score','sentence'])
keys_df['score'] = keys_df['score'].astype(float)

# ordered by relevance 
keys_df.sort_values(['sentence','score']).head(30)

,keywords,begin,end,score,sentence
0,allergies,69,77,0.224708,0
1,used,84,87,0.184242,1
2,allergies,97,105,0.224708,1
3,seattle,125,131,0.536178,1
22,allergies,719,727,0.224708,10
23,weight,749,754,0.687564,11
25,without,840,846,0.428513,12
24,erythematous,827,838,0.428628,12
27,erythematous,874,885,0.428628,13
26,nasal,857,861,0.430840,13


### Keyword dari Dataframe

In [11]:
df = spark.read\
                .option("header", "true")\
                .csv("mtsamples.csv")\
                
df.show(truncate=50)

+--------------------+--------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|      Type/Specialty|                                 Sample Name|                                       Description|                                        Transcript|                                          Keywords|
+--------------------+--------------------------------------------+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|Allergy / Immunology|                           Allergic Rhinitis|A 23-year-old white female presents with compla...|SUBJECTIVE: This 23-year-old white female prese...|allergy / immunology, allergic rhinitis, allerg...|
|Allergy / Immunology|                  Allergy Evaluation Consult|Acute allergic reaction, etiology uncertain, ho..

In [12]:
df = df.select('Transcript')

In [13]:
result = yake_pipeline.fit(df).transform(df)

In [14]:
result = result.withColumn('unique_keywords', F.array_distinct("keywords.result"))

In [15]:
result = result.withColumn('unique_keywords', F.array_distinct("keywords.result"))
def highlight(text, keywords):
    for k in keywords:
        text = (re.sub(r'(\b%s\b)'%k, r'<span style="background-color: yellow;">\1</span>', text, flags=re.IGNORECASE))
    return text

In [16]:
highlight_udf = udf(highlight, StringType())

In [17]:
result = result.withColumn("highlighted_keywords",highlight_udf('Transcript','unique_keywords'))

In [18]:
for r in result.select("highlighted_keywords").limit(20).collect():
    display(HTML(r.highlighted_keywords))
    print("\n\n")

Traceback (most recent call last):                                  (0 + 1) / 1]
  File "/Users/mhihsan/miniconda3/envs/kuliah/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/mhihsan/miniconda3/envs/kuliah/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/mhihsan/miniconda3/envs/kuliah/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/mhihsan/miniconda3/envs/kuliah/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError
